In [1]:
import glob
import geopandas as gpd
import numpy as np
import pandas as pd
import json
from datetime import datetime

cells = gpd.read_file('global-inputs/HR6-cells-beach-slope.gpkg')


def apply_tidal_correction(cell_id):
    slope = cells.loc[cells['cell_id'] == cell_id, 'beach_slope'].values[0]
    ref_ele = 0
    folder = f"data/HR6/{cell_id}"
    raw_df = pd.read_csv(f"{folder}/cell_timeseries.csv")
    # drop rows where IW_CDI exceeds 10 & EOV_CDI exceeds 5
    raw_df.drop(raw_df[(raw_df['IW_CDI'] > 10) | (raw_df['IW_CDI'] < -10) ].index, inplace=True)
    raw_df.drop(raw_df[(raw_df['EOV_CDI'] > 5) | (raw_df['EOV_CDI'] < -10) ].index, inplace=True)

    raw_df['date'] = pd.to_datetime(raw_df['date'])

    tide_df = pd.read_json(f"{folder}/image_metadata.json")
    tide_df['date'] = tide_df['image_date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))

    merged_df = pd.merge(raw_df, tide_df[['date', 'tide_level_msl']], on='date', how='left') # Merge raw_df and tide_df on the date column
    
    if np.isnan(slope):
        merged_df['corrected_IW_shoreline_position'] = (merged_df.IW_shoreline_chg).round(2)
        merged_df['normalised_IW_shoreline_position'] = (merged_df['corrected_IW_shoreline_position'] - merged_df['corrected_IW_shoreline_position'].iloc[0]).round(2)
        merged_df['normalised_EOV_shoreline_position'] = (merged_df['EOV_shoreline_chg'] - merged_df['EOV_shoreline_chg'].iloc[0]).round(2)
        merged_df['corrected_for_tide'] = False
    else:
        merged_df['tidal_correction'] = (merged_df.tide_level_msl-ref_ele)/slope
        merged_df['corrected_IW_shoreline_position'] = (merged_df.IW_shoreline_chg + merged_df.tidal_correction).round(2)
        merged_df['normalised_IW_shoreline_position'] = (merged_df['corrected_IW_shoreline_position'] - merged_df['corrected_IW_shoreline_position'].iloc[0]).round(2)
        merged_df['normalised_EOV_shoreline_position'] = (merged_df['EOV_shoreline_chg'] - merged_df['EOV_shoreline_chg'].iloc[0]).round(2)
        merged_df['corrected_for_tide'] = True

        merged_df.drop_duplicates(subset=['date'], keep='first', inplace=True)
    
    return merged_df

def return_landcover_percentage_change(df):
    landcover = df.drop(columns='date').copy()
    df['total_area'] = landcover.sum(axis=1)
    df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
    df['water_area_percentage'] = (df['water_area (Ha)']/df['total_area'])*100
    df['vegetation_area_percentage'] = (df['vegetation_area (Ha)']/df['total_area'])*100

    df[['sand_area_percentage_change', 'water_area_percentage_change', 'vegetation_area_percentage_change']] = df[['sand_area_percentage', 'water_area_percentage', 'vegetation_area_percentage']].diff().round(2)

    df.drop_duplicates(subset=['date'], keep='first', inplace=True)
    
    return df[['date', 'sand_area_percentage_change', 'water_area_percentage_change', 'vegetation_area_percentage_change']].fillna(0)


def return_correct_tide(metadata_file):
    df = pd.read_json(metadata_file)
    if 'tide_level_msl_x' in df.columns:
        df.drop(columns=['tide_level_msl_x'], inplace=True)
        df.rename(columns={'tide_level_msl_y': "tide_level_msl"}, inplace=True)
        df['tide_level_msl'] = df["tide_level_msl"].round(2)
        #write metadata to file
        meta_dict = df.to_dict(orient='list') 
        with open(metadata_file, 'w') as file:
            file.write(json.dumps(meta_dict, indent=4))
    else:
        return

In [2]:
cell_ids = cells.cell_id.tolist()

failed = []

for cell in cell_ids:
    try:
        corrected_df = apply_tidal_correction(cell)
        shorelines_df = corrected_df[['date', 'normalised_IW_shoreline_position', 'normalised_EOV_shoreline_position', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg']]
        shorelines_df.rename(columns={'EOV_shoreline_chg': 'EOV_shoreline_position'})
        shorelines_df.to_csv(f"data/HR6/{cell}/shoreline_timeseries_tidal_correction.csv")

        landcover_df = corrected_df[['date', 'sand_area (Ha)', 'water_area (Ha)', 'vegetation_area (Ha)']]
        landcover_df = return_landcover_percentage_change(landcover_df)
        landcover_df.to_csv(f"data/HR6/{cell}/landcover_timeseries_percentage_change.csv")
    except:
        failed.append(cell)

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

...

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

/tmp/ipykernel_1631553/1772916063.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_area'] = landcover.sum(axis=1)
/tmp/ipykernel_1631553/1772916063.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
/tmp/ipykernel_1631553/1772916063.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati